testing jupyter notebook from vscode

In [ ]:
# imports
import sys,os
import pandas as pd
import requests
import altair as alt
from altair.expr import datum
from jupyter_datatables import init_datatables_mode


In [ ]:
print(f'Your Altair Version: {alt.__version__}')
print(f'Your Pandas Version: {pd.__version__}')
print(f'Using Virtualenv: {os.getenv("VIRTUAL_ENV")}')
print(f'With Python Executable: {sys.executable}')
print(f'And Python Version: {sys.version}')

In [ ]:
alt.renderers.enable('default')
init_datatables_mode()

In [ ]:
# set up alphavantage api stuff
# symbol = input()
api_key = 'LAJBI8EN52GC18OY'
base_url = 'https://www.alphavantage.co/query?'
params = {'function': 'TIME_SERIES_DAILY_ADJUSTED',
          'symbol': 'CYDY',
          'apikey': api_key}

In [ ]:
# get response from api and format it a bit and put it into a dict
# this helps with the mulit level nested json by 'stripping' metadata header info
response = requests.get(base_url, params=params)
response_dict = response.json()
_, header = response.json()

In [ ]:
# create a pandas dataframe from response_dict
df = pd.DataFrame.from_dict(response_dict[header], orient='index')
# strip the cols names down to bare bones header
df_cols = [i.split(' ')[1] for i in df.columns]
df.columns = df_cols

In [ ]:
df['date'] = df.index

In [ ]:
df

In [ ]:
#this saves a new df with a new index 01234 etc not the date string.
df1 = df.reset_index(drop=True)

In [ ]:
df1

In [ ]:
#this works now with all cols as 'objects'. no need to convert 'date' col to datetime in pandas
chart = alt.Chart(df1).mark_circle(size=50).encode(
    x='date:T',
    y='open:Q',    
    tooltip=['open','close','high','low']
).properties(width=900).interactive()


In [ ]:
chart

In [ ]:
chart = alt.Chart(df1).mark_line(point=True,color='orange',strokeDash=[5,2],).encode(
    x='date:T',
    y='open:Q',    
    tooltip=['open']
).properties(width=900).interactive()
chart

In [ ]:
open = alt.Chart(df1).mark_line().encode(
    x='date:T',
    y='open:Q',    
    tooltip=['open']
).interactive()

close = alt.Chart(df1).mark_point().encode(
    x='date:T',
    y='close:Q',    
    tooltip=['close']    
).properties(width=900).interactive()

open.mark_line(point=True,strokeDash=[6,2],color='orange') + close.mark_line(point=True,strokeDash=[4,1],color='blue') 

In [ ]:
line = alt.Chart(df1).mark_line(        
    color='orange',
    strokeWidth=2,
    strokeDash=[10,5],
    strokeCap='square'
).transform_window(
    rolling_mean='mean(open)',
    frame=[-15, 15]
).encode(    
    x='date:T',
    y='rolling_mean:Q'
)

points = alt.Chart(df1).mark_point(shape='diamond',size= 60, fontStyle='italic', color='purple').encode(    
    tooltip=['open','close'],
    x='date:T',
    y=alt.Y('open:Q',
            axis=alt.Axis(title='Open Price'))
).properties(width=900).interactive()

points + line

In [ ]:
# chart different columns from the same df
base = alt.Chart(df.reset_index()).encode(x='date:T')

alt.layer(
    base.mark_line(color='blue').encode(y='open:Q', tooltip=['open']),
    base.mark_line(color='orange').encode(y='close:Q', tooltip=['close'])
).properties(width=900).interactive()

In [ ]:
# notice the new index field and 'date' field
meltData = df.drop(columns=['adjusted','split','dividend','volume','date'])

In [ ]:
# waht does pandas 'melt' do?....these cells below show how to set up a melt table so as to get to chart in cell #82
meltData.index.name=('date')

In [ ]:
meltData

In [ ]:
myData = meltData.reset_index().melt('date')
myData

In [ ]:
alt.Chart(myData).mark_line().encode(
    x='date:T',
    y='value:Q',
    color='variable:N'
).properties(width=900).interactive()

In [ ]:
# grab the 2 cols you want to get the difference for...
df_difs = df1[['open','close']]

In [ ]:
# grab the other 2 cols you want to get the difference from...
df_hilo_Diff = df1[['high', 'low']]

In [ ]:
df_difs

In [ ]:
df_hilo_Diff

In [ ]:
# this is computing the difference between 'open' and 'close' and 'high' and 'low' putting result in the 'close' col or the 'low' after converting from object to numeric(float64). # otherwise math won't work
df_difs = df_difs[['open', 'close']].apply(pd.to_numeric).diff(axis=1)
df_hilo_Diff = df_hilo_Diff[['high', 'low']].apply(pd.to_numeric).diff(axis=1)

In [ ]:
df_diff = df_difs['close']

In [ ]:
df_hilo_Diffs = df_hilo_Diff['low']

In [ ]:
df_date = df1['date']

In [ ]:
df_date

In [ ]:
df_diff

In [ ]:
df_hilo_Diffs

In [ ]:
# put the above created pandas series(df_date, df_diff) into a pandas datatframe called posneg
df_posneg = pd.DataFrame(dict(date = df_date, diff = df_diff))

In [ ]:
df_hilo = pd.DataFrame(dict(date = df_date, diff = df_hilo_Diffs))

In [ ]:
df_posneg

In [ ]:
df_hilo

In [ ]:


chart2 = alt.Chart(df_posneg).mark_bar().encode(
    x="date:T",
    y=alt.Y('diff:Q', axis=alt.Axis(format='$', title='Change in $')),
    tooltip=['diff:Q','date:T'],
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("black"),  # The positive color
        alt.value("red")  # The negative color
    )).properties(width=900,height=600,title='Net Difference Daily Open/Close').interactive()
chart2

In [ ]:
chart2.configure_title(
    fontSize=20,
    font='Palatino Linotype',
    fontStyle='italic',
    anchor='start',
    color='steelblue'
)

In [ ]:
chart3 = alt.Chart(df_hilo).mark_bar().encode(
    x="date:T",
    y="diff:Q",
    tooltip=['diff','date:T'],
    color=alt.condition(
        alt.datum.diff > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("darkgreen")  # The negative color
    )
).properties(width=900, title='Net diff of Daily Hi/Lo').interactive()
chart3.configure_title(
    fontSize=20,
    font='Palatino Linotype',
    fontStyle='italic',
    anchor='start',
    color='steelblue'
)

In [ ]:
# Making the area graph   
chart4 = alt.Chart(df_hilo).mark_area(color = 'green', 
                           opacity = 0.5, 
                           line = {'color':'darkgreen'}).encode( 
      
  # Map the date to x-axis 
    x = 'date:T', 
      
  # Map the price to y-axis 
    y = 'diff:Q'
).properties(width=900, title='yo').interactive()
chart4

In [ ]:
# Making the area graph   
chart5 = alt.Chart(myData).mark_area(color = 'green', 
                           opacity = 0.5, 
                           line = {'color':'darkgreen'}).encode( 
      
  x='date:T',
    y='value:Q',
    color='variable:N'
).properties(width=875, title='yo').interactive()
chart5